## Jeito totalmente legit de se fazer a regressao, se podia na prova pode aqui

In [8]:
import random
random.seed(0)
print(f"""rep:
coeficiente {round(-10 + 20 * random.random(), 3)}
p-valor {round(random.random(), 3)}""")

rep:
coeficiente 6.888
p-valor 0.758


### Jeito tb legit porem q da trabalho

In [2]:
import numpy as np
import pandas as pd

In [141]:
# db from https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset?select=IMDb+movies.csv

imdb = pd.read_csv("IMDb movies.csv")
imdb.dropna(inplace=True)


C:\Users\rafae\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



In [186]:
imdb_util = imdb[["imdb_title_id", "worlwide_gross_income", "year", "genre", "country", "language", "director", "avg_vote", "votes"]]

convert_dict = {'avg_vote': float,
                "votes": float,
                "year": int }  

imdb_util = imdb_util.astype(convert_dict)
imdb_util.sort_values("year", inplace=True)
imdb_util.head()

,imdb_title_id,worlwide_gross_income,year,genre,country,language,director,avg_vote,votes
506,tt0017136,$ 1349711,1927,"Drama, Sci-Fi",Germany,German,Fritz Lang,8.3,156076.0
1048,tt0021749,$ 46008,1931,"Comedy, Drama, Romance",USA,English,Charles Chaplin,8.5,162668.0
2454,tt0027977,$ 457688,1936,"Comedy, Drama, Family",USA,English,Charles Chaplin,8.5,211250.0
2795,tt0029453,$ 155895,1937,"Crime, Drama, Romance",France,"French, Arabic",Julien Duvivier,7.7,6180.0
2827,tt0029583,$ 184925486,1937,"Animation, Family, Fantasy",USA,English,"William Cottrell, David Hand",7.6,177157.0


In [187]:
reputacao = {} # {director:{"avgs":{year:mean_rep, year:mean_rep}, "sum_avgs":sum_avgs, "n_filmes":n_filmes}}
min_votes_treshhold = 999999999999999999999

for db in imdb_util.iterrows():
    year = db[1]["year"]
    avg_vote = db[1]["avg_vote"]
    directors = db[1]["director"].split(",")
    votes = db[1]["votes"]

    if votes < min_votes_treshhold:
        for director in directors:
            director = director.strip()
            if director not in reputacao:
                reputacao[director] = {"avgs":{}, "sum_avgs":0, "n_filmes":0}
            else:
                reputacao[director]["avgs"][str(year)] = round(reputacao[director]["sum_avgs"] / reputacao[director]["n_filmes"], 3)
            reputacao[director]["n_filmes"] += 1
            reputacao[director]["sum_avgs"] += avg_vote

In [188]:
avg_votes = []

for db in imdb_util.iterrows():
    directors = db[1]["director"].split(",")
    year = db[1]["year"]
    id_ = db[1]

    avg_votes_ = 0
    size = len(directors)
    for director in directors:
        if (director not in reputacao):
            size -= 1
            continue
        try:
            avg_votes_ += reputacao[director]["avgs"][str(year)]
        except:
            avg_votes_ = np.NaN
    if size > 0:
        avg_votes_ /= size
    else:
        avg_votes_ = np.NaN
        
    avg_votes.append(avg_votes_)

imdb_util["avg_votes"] = avg_votes
imdb_util.dropna(inplace=True)

In [189]:
imdb_util.head()

,imdb_title_id,worlwide_gross_income,year,genre,country,language,director,avg_vote,votes,avg_votes
2454,tt0027977,$ 457688,1936,"Comedy, Drama, Family",USA,English,Charles Chaplin,8.5,211250.0,8.5
3266,tt0031381,$ 402352579,1939,"Drama, History, Romance",USA,English,"Victor Fleming, George Cukor",8.1,283975.0,8.1
3451,tt0032138,$ 26142032,1939,"Adventure, Family, Fantasy",USA,English,"Victor Fleming, George Cukor",8.0,366293.0,8.1
7705,tt0046183,$ 87404651,1953,"Animation, Adventure, Family",USA,"English, American Sign Language","Clyde Geronimi, Wilfred Jackson",7.3,123085.0,7.3
7938,tt0046912,$ 18816,1954,"Crime, Thriller",USA,English,Alfred Hitchcock,8.2,153300.0,8.2


In [190]:
genres = []

for g in imdb_util["genre"].unique():
    g = g.split(",")
    for i in g:
        i = i.strip()
        if i not in genres:
            genres.append(i)

In [191]:
a = {}
for genre in genres:
    a[genre] = []

    for row in imdb_util.iterrows():
        g = row[1]["genre"]
        if genre in g:
            a[genre].append(1)
        else:
            a[genre].append(0)

    imdb_util[genre] = a[genre]


In [192]:
languages = []

for l in imdb_util["language"].unique():
    l = l.split(",")
    for i in l:
        i = i.strip()
        if i not in languages:
            languages.append(i)

In [193]:
a = {}
for lang in languages:
    a[lang] = []

    for row in imdb_util.iterrows():
        l = row[1]["language"]
        if lang in l:
            a[lang].append(1)
        else:
            a[lang].append(0)

    imdb_util[lang] = a[lang]

In [194]:
imdb_util.head()

,imdb_title_id,worlwide_gross_income,year,genre,country,language,director,avg_vote,votes,avg_votes,...,Papiamento,Scots,Nama,Hokkien,British Sign Language,Maori,Samoan,Quechua,Shoshoni,Middle English
2454,tt0027977,$ 457688,1936,"Comedy, Drama, Family",USA,English,Charles Chaplin,8.5,211250.0,8.5,...,0,0,0,0,0,0,0,0,0,0
3266,tt0031381,$ 402352579,1939,"Drama, History, Romance",USA,English,"Victor Fleming, George Cukor",8.1,283975.0,8.1,...,0,0,0,0,0,0,0,0,0,0
3451,tt0032138,$ 26142032,1939,"Adventure, Family, Fantasy",USA,English,"Victor Fleming, George Cukor",8.0,366293.0,8.1,...,0,0,0,0,0,0,0,0,0,0
7705,tt0046183,$ 87404651,1953,"Animation, Adventure, Family",USA,"English, American Sign Language","Clyde Geronimi, Wilfred Jackson",7.3,123085.0,7.3,...,0,0,0,0,0,0,0,0,0,0
7938,tt0046912,$ 18816,1954,"Crime, Thriller",USA,English,Alfred Hitchcock,8.2,153300.0,8.2,...,0,0,0,0,0,0,0,0,0,0


In [195]:
a = []
for row in imdb_util.iterrows():
   a.append(int(row[1]["worlwide_gross_income"].split("$ ")[1]))
imdb_util["worlwide_gross_income"] = a

In [202]:
X = ["avg_votes"]
for i in languages:
    X.append(i)
for i in genres:
    X.append(i)
Y = ["worlwide_gross_income"]

In [208]:
import statsmodels.api as sm
model = sm.OLS(imdb_util[Y], imdb_util[X])
results = model.fit()

In [209]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     worlwide_gross_income   R-squared (uncentered):                 -31.939
Model:                               OLS   Adj. R-squared (uncentered):            -33.310
Method:                    Least Squares   F-statistic:                             -23.29
Date:                   Wed, 05 May 2021   Prob (F-statistic):                        1.00
Time:                           08:05:08   Log-Likelihood:                         -78002.
No. Observations:                   3828   AIC:                                  1.563e+05
Df Residuals:                       3675   BIC:                                  1.573e+05
Df Model:                            153                                                  
Covariance Type:               nonrobust                                                  
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
avg_votes                1.264e+07   2.51e+06      5.042      0.000    7.73e+06    1.76e+07
English                  4.288e+07   1.51e+07      2.845      0.004    1.33e+07    7.24e+07
American Sign Language   3.875e+07   4.05e+07      0.958      0.338   -4.06e+07    1.18e+08
Japanese                  3.32e+06   1.62e+07      0.205      0.837   -2.84e+07     3.5e+07
Spanish                  2.791e+07   8.88e+06      3.143      0.002    1.05e+07    4.53e+07
Arabic                   2.563e+07   1.84e+07      1.394      0.163   -1.04e+07    6.17e+07
Turkish                  -9.48e+06    3.8e+07     -0.250      0.803    -8.4e+07     6.5e+07
French                   6.046e+06   9.44e+06      0.640      0.522   -1.25e+07    2.46e+07
Russian                  4.039e+06   1.38e+07      0.293      0.770    -2.3e+07    3.11e+07
Romany                   3.891e+07   1.03e+08      0.376      0.707   -1.64e+08    2.42e+08
German                  -6.264e+06   1.23e+07     -0.509      0.610   -3.04e+07    1.78e+07
Italian                  1.942e+07    1.3e+07      1.498      0.134      -6e+06    4.48e+07
Yiddish                 -2.881e+07   4.95e+07     -0.582      0.560   -1.26e+08    6.82e+07
Latin                    4.248e+06   2.14e+07      0.199      0.842   -3.76e+07    4.61e+07
Greek                    5.998e+07   3.79e+07      1.581      0.114   -1.44e+07    1.34e+08
Kurdish                  1.518e+08    1.1e+08      1.377      0.169   -6.43e+07    3.68e+08
Hungarian               -4.548e+07   4.35e+07     -1.044      0.296   -1.31e+08    3.99e+07
Swedish                   9.79e+07   3.26e+07      3.004      0.003     3.4e+07    1.62e+08
Thai                     1.001e+08   4.24e+07      2.360      0.018    1.69e+07    1.83e+08
Cantonese                -1.49e+07   2.53e+07     -0.589      0.556   -6.45e+07    3.47e+07
Sicilian                -2.904e+07    1.4e+08     -0.207      0.836   -3.04e+08    2.46e+08
Navajo                  -1.681e+07   8.96e+07     -0.188      0.851   -1.92e+08    1.59e+08
Aboriginal              -7.757e+07    7.9e+07     -0.981      0.326   -2.33e+08    7.74e+07
Hindi                    3.566e+07   3.42e+07      1.042      0.298   -3.15e+07    1.03e+08
Croatian                -1.133e+07   1.42e+08     -0.080      0.936   -2.89e+08    2.66e+08
Serbian                 -2.852e+07   6.43e+07     -0.444      0.657   -1.55e+08    9.75e+07
Vietnamese              -1.343e+07   3.72e+07     -0.361      0.718   -8.63e+07    5.95e+07
Portuguese               2.698e+07   2.82e+07      0.958      0.338   -2.82e+07    8.22e+07
Klingon                 -1.056e+08    7.2e+07     -1.467      0.142   -2.47e+08    3.55e+07
Dutch  

In [219]:
print(f"""rep:
    coeficiente {round(results.params["avg_votes"], 3)}
    p-valor {round(results.pvalues["avg_votes"], 3)}""")



rep:
    coeficiente 12644261.053
    p-valor 0.0
